In [1]:
import pandas as pd

In [3]:
path1 = '/Users/Mark1/Documents/Data Science/preppin_data/2024/week_2/data/input/PD 2024 Wk 1 Output Flow Card.csv'
path2 = '/Users/Mark1/Documents/Data Science/preppin_data/2024/week_2/data/input/PD 2024 Wk 1 Output Non-Flow Card.csv'

In [4]:
df1 = pd.read_csv(path1)
df2 = pd.read_csv(path2)

print(df1.head())
print(df2.head())

         Date Flight Number      From        To            Class   Price  \
0  22/07/2024         PA010     Tokyo  New York          Economy  2380.0   
1  20/04/2024         PA002  New York    London          Economy  3490.0   
2  23/01/2024         PA010     Tokyo  New York  Premium Economy   825.0   
3  05/06/2024         PA006     Tokyo    London      First Class   618.0   
4  30/03/2024         PA004     Perth    London      First Class   446.0   

  Flow Card?  Bags Checked   Meal Type  
0        Yes             0    Egg Free  
1        Yes             1       Vegan  
2        Yes             1  Vegetarian  
3        Yes             3       Vegan  
4        Yes             1    Nut Free  
         Date Flight Number      From        To            Class   Price  \
0  28/09/2024         PA008     Perth  New York          Economy  1855.0   
1  01/10/2024         PA008     Perth  New York   Business Class   634.8   
2  04/03/2024         PA007  New York     Perth   Business Class   45